In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import colors
# from PIL import Image
import pandas as pd
import collections
from linecache import getline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import pickle
import seaborn as sn
plt.style.use('ggplot')


%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20,10)


In [ ]:
np.random.seed(10)

In [ ]:
# Venice Water Level Dataset from 1983 to 2015
dataframe = pd.read_csv('./lstm-flood/dataset/venezia.csv', usecols=[1], engine='python', skipfooter=3)
dataset   = dataframe.values
dataset   = dataset.astype('float32')
dataframe.head()


In [ ]:
plt.plot(dataset)

In [ ]:
water_levels = dataframe['level'].values.tolist()

In [ ]:
cleaned_levels = []
                 
for i in water_levels:
    if i >= 0:
        cleaned_levels.append(i)

In [ ]:
df_clean = pd.DataFrame(cleaned_levels)

In [ ]:
plt.plot(df_clean)

In [ ]:
# normalize the dataset
scaler  = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets

percentage_train = 0.80

train_size  = int(len(dataset) * percentage_train)
test_size   = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print("Training Datapoints :", len(train), "testing Datapoints :", len(test))

In [ ]:
print('--- Training Data ---', percentage_train * 100, '% ---')
plt.plot(train)
plt.show()
print('--- Test Data ---', 100-(percentage_train * 100), '% ---')
plt.plot(test)
plt.show()

In [ ]:
# This function creates a sliding window of the dataset.
# applying a window or batch of dataset make batched features dataset
# converting a problem in a nested loop, to reduce the time complexity.


"""
LSTMs do not require a sliding window of inputs. They can remember what they have seen in the past, and if you feed in training examples one at a time they will choose the right size window of inputs to remember on their own.

LSTM's are already prone to overfitting, and if you feed in lots of redundant data with a sliding window then yes, they are likely to overfit.
On the other hand, a sliding window is necessary for time series forecasting with Feedforward Neural Networks, because FNNs require a fixed size input and do not have memory, so this is the most natural way to feed them time series data.
Whether or not the FNN will overfit depends on its architecture and your data, but all standard regularization techniques will apply if it does. For example you can try choosing a smaller network, L2 regularization, Dropout, etc.
"""
# in simple words cutting down the data into chunks of data.
def create_dataset(dataset, sliding_window=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-sliding_window-1):
        a = dataset[i:(i+sliding_window), 0]
        dataX.append(a)
        dataY.append(dataset[i + sliding_window, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# use a n-24 sliding window equivalent to 24 hours of historical data - original 10
slide_window   = 24
trainX, trainY = create_dataset(train, slide_window)
testX, testY   = create_dataset(test, slide_window)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX  = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
np.shape(testX)
print(testX)

In [ ]:
#Setup the LSTM

model = Sequential()
model.add(LSTM(4, input_dim=slide_window))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
# Epoch size 50
model.fit(trainX, trainY, nb_epoch=50, batch_size=1, verbose=2)

In [ ]:
# Print out the evaluation for both the
import math
trainScore = model.evaluate(trainX, trainY, verbose=0)
trainScore = math.sqrt(trainScore)
trainScore = scaler.inverse_transform(np.array([[trainScore]]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = model.evaluate(testX, testY, verbose=0)
testScore = math.sqrt(testScore)
testScore = scaler.inverse_transform(np.array([[testScore]]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(model) 
  
# Load the pickled model 
model_from_pickle = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions 
model_from_pickle.predict(testX)

In [ ]:
to_predict_list = [1] * 24
# print(to_predict_list)
to_predict = np.array(to_predict_list).reshape(1, 1, 24)
print(to_predict)
loaded_model = pickle.load(open("./filename.pkl", "rb"))
result = loaded_model.predict(to_predict)
result = str(result[0][0])
print(result)